In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sqlalchemy import create_engine
from sklearn import metrics

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'heartdisease'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

engine.dispose()

heartdisease_df = pd.read_sql_query('select * from heartdisease',con=engine)
heartdisease_df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
5,56.0,1.0,2.0,120.0,236.0,0.0,0.0,178.0,0.0,0.8,1.0,0.0,3.0,0
6,62.0,0.0,4.0,140.0,268.0,0.0,2.0,160.0,0.0,3.6,3.0,2.0,3.0,3
7,57.0,0.0,4.0,120.0,354.0,0.0,0.0,163.0,1.0,0.6,1.0,0.0,3.0,0
8,63.0,1.0,4.0,130.0,254.0,0.0,2.0,147.0,0.0,1.4,2.0,1.0,7.0,2
9,53.0,1.0,4.0,140.0,203.0,1.0,2.0,155.0,1.0,3.1,3.0,0.0,7.0,1


In [3]:
# Define the features and the outcome
X = heartdisease_df.iloc[:, :13]
y = heartdisease_df.iloc[:, 13]

# Replace missing values (marked by ?) with a 0
X = X.replace(to_replace='?', value=0)

# Binarize y so that 1 means heart disease diagnosis and 0 means no diagnosis
y = np.where(y > 0, 0, 1)

In [4]:
scaler=StandardScaler()
X_std =scaler.fit_transform(X)

In [35]:
gmm_cluster= GaussianMixture(n_components=2)

clusters= gmm_cluster.fit_predict(X_std)

print('ARI Score: {}'.format(adjusted_rand_score(y, clusters)))
print('Silhouette Score: {}'.format(silhouette_score(X_std, clusters, metric='manhattan')))

ARI Score: 0.4207322145049338
Silhouette Score: 0.2277053339988964


K-means produced ARI and silhouette scores of 0.43 and 0.17, respectively, while hierarchical clustering produced solutions of 0.37 and 0.23 via complete linkage. GMM produced a well-rounded solution which may not have beaten other solutions by a single metric, but performs well with or without labels. 

In [54]:
covariance= ['full', 'tied', 'diag', 'spherical']

for cov in covariance: 
    gmm_cluster= GaussianMixture(n_components=2, covariance_type=cov)
    clusters=gmm_cluster.fit_predict(X_std)
    print('ARI score for GMM solution with ' +str(cov)+ ' covariance type: {}'.format(adjusted_rand_score(y, clusters)))
    print('Silhouette score for GMM solution with ' +str(cov)+ ' covariance type: {}\n'.format(silhouette_score(X_std, clusters)))

ARI score for GMM solution with full covariance type: 0.4207322145049338
Silhouette score for GMM solution with full covariance type: 0.16118591340148433

ARI score for GMM solution with tied covariance type: 0.46482432589803474
Silhouette score for GMM solution with tied covariance type: 0.16607012124631093

ARI score for GMM solution with diag covariance type: 0.37878842718089933
Silhouette score for GMM solution with diag covariance type: 0.15836933745078685

ARI score for GMM solution with spherical covariance type: 0.20765243525722465
Silhouette score for GMM solution with spherical covariance type: 0.12468753110276876



The _tied_ covariance type performs best with a 0.46 ARI score and 0.167 silhouette score. 